In [2]:
%reload_ext autoreload
%autoreload 2

import socket
import psutil
import sys 
import os
from typing import Any
from functools import partial
import json
from pprint import pprint
from collections import OrderedDict
from datetime import datetime
import os
import glob

# lib_path = ['/fsx_0/user/tranx/experiments']
import_paths = [
    # "/fsx_0/user/tranx/experiments",
    # "/fsx_0/user/tranx",
    # "/fsx_0/user/tranx/rsync",
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/experiments/aws",
    # "/fsx_0/user/tranx/experiments/lib"
]
for path in import_paths:
    if path not in sys.path:
        sys.path.append(path)
        
from launch_evals import read_eval_results

import pandas as pd
pd.set_option('display.max_rows', 100)

def get_trainer_state(output_dir: str):
    checkpoint_dirs = glob.glob(f"{output_dir}/checkpoint-*")
    checkpoints = [int(d.split("/")[-1].split("-")[-1]) for d in checkpoint_dirs]
    if len(checkpoints) == 0:
        return None, None 
    
    checkpoint_latest = max(checkpoints)
    # print(f"{checkpoint_latest=}")

    trainer_state_file = os.path.join(
        output_dir,
        f"checkpoint-{checkpoint_latest}",
        "trainer_state.json"
    )

    with open(trainer_state_file, 'r') as file:
        state = json.load(file)
        
    return checkpoint_latest, state

# SFT

In [11]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_SFT/MM10.2_SFT_ViTG_392px_vqa",
    read_format="report_mmai",
)

,Average,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE
200,69.378333,NaN,NaN,49.4,81.51,68.49,NaN,76.01,70.92,69.94,NaN,NaN


# Stage 2

In [12]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTG_392px_stage2_R4a_n128",
    base_eval_output_dir="evals_master",
    read_format="report_mmai",
)

here


,Average,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE
12600,70.704172,58.11,84.845893,49.9,81.90,61.17,79.26,75.14,65.16,60.73,76.34,85.19
12800,70.282528,57.44,81.877811,49.2,81.93,62.53,79.27,75.01,63.64,59.93,77.24,85.04
13000,69.910923,57.00,80.170156,49.3,82.12,61.43,79.23,75.46,63.27,58.62,77.32,85.10
13200,69.97723,58.56,79.649528,49.6,82.16,61.98,79.31,75.77,62.33,58.15,77.32,84.92
13400,69.444772,58.78,78.452494,49.4,80.89,62.15,79.16,75.51,61.31,56.09,77.21,84.94
13600,69.129349,58.89,77.652835,49.9,80.60,61.83,79.25,75.56,60.36,54.52,76.72,85.14
13800,69.109941,59.56,77.479356,49.8,80.34,62.21,79.31,75.60,59.43,54.77,76.79,84.92


In [10]:
# R2
df = read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTG_392px_stage2_R4a_n128",
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTH_336px_stage2_R4_n128",
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTH_336px_unfreeze_stage2_R2_n128",
    base_eval_output_dir="evals_master",
    # read_format="report_mmai",
    start_step=0
    # steps=[5800],
)

In [11]:
docvqa = []
for k in df.columns:
    if "docvqa" in k:
        print(k)
        docvqa.append(k)
df[docvqa]

eval_docvqa/anls_total_score
eval_docvqa/mllm_evaluation_anls_score
eval_docvqa/mmllm_fixed_anls_score


,eval_docvqa/anls_total_score,eval_docvqa/mllm_evaluation_anls_score,eval_docvqa/mmllm_fixed_anls_score
12600,0.779557,0.776888,0.848459
12800,0.752117,0.751883,0.818778
13000,0.726573,0.726567,0.801702
13200,0.710580,0.711208,0.796495
13400,0.684102,0.683218,0.784525
13600,0.670301,0.669707,0.776528
13800,0.661797,0.663069,0.774794


In [5]:
# R1
read_eval_results(
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTG_392px_stage2_R1_n128",
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTH_336px_stage2_R1_n128",
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage2_70B/MH24_70B_ViTH_336px_unfreeze_stage2_R1_n128",
    base_eval_output_dir="evals_master",
    read_format="report_mmai",
    start_step=4000
)

,Average,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
4000,58.944444,58.33,NaN,41.4,65.45,47.07,76.01,73.41,50.38,43.97,74.48,NaN,NaN,NaN
4200,61.251111,NaN,75.74,41.1,66.35,47.86,76.02,73.24,51.32,44.85,74.78,NaN,NaN,NaN
4400,60.736,56.67,75.31,41.4,65.61,48.15,75.89,73.60,50.79,45.32,74.62,NaN,NaN,NaN


# Stage 1

In [3]:
# base_dir = "/fsx_3/bucket/tranx/checkpoints/vev0"
base_dir = "/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B"
output_dirs = glob.glob(f"{base_dir}/*")
for d in output_dirs:
    if "bz48" in d:
        print(d)
        name = d.split("/")[-1]
        checkpoint_latest, state = get_trainer_state(d)
        
        print(f"{name}, {checkpoint_latest=}")
        # print(f"{name}")

/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_ViTG_392px_mt8400_R2_bz48
LLama3.1_70B_ViTG_392px_mt8400_R2_bz48, checkpoint_latest=3000
/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_mt8400_R2_bz48
MH24_70B_ViTG_392px_mt8400_R2_bz48, checkpoint_latest=15000
/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_BigG_336_R2_bz48
LLama3.1_70B_BigG_336_R2_bz48, checkpoint_latest=4800
/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_mt8400_R2_bz48_resume
MH24_70B_ViTG_392px_mt8400_R2_bz48_resume, checkpoint_latest=18600
/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTH_336px_R2_bz48
MH24_70B_ViTH_336px_R2_bz48, checkpoint_latest=12900
/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_mt8400_R2_layer1_bz48
MH24_70B_ViTG_392px_mt8400_R2_layer1_bz48, checkpoint_latest=6700
/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_ViTG_392px_mt8400_layer1_R2_bz48
LLama3.1_70B_ViTG_392px_mt8400_layer1_R2_bz48, checkpoint_latest=28

In [22]:
read_eval_results(
    watch_path="/fsx_3/bucket/tranx/checkpoints/vev0/Llama3.1_8B_ViTG_392_warmup_mlp_2x2_resume",
    # base_eval_output_dir="evals_master",
    read_format="report_mmai",
    # steps=[7500]
)

,Average,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
500,51.999,47.12,63.80,31.5,50.32,41.80,68.08,61.17,47.71,42.32,66.17,NaN,NaN,NaN
1000,51.843,46.46,63.89,30.6,50.16,41.70,68.05,61.12,48.01,42.39,66.05,NaN,NaN,NaN
1500,51.699,45.58,63.93,30.8,50.06,41.35,68.10,61.10,48.26,42.00,65.81,NaN,NaN,NaN
2000,51.983,47.46,63.84,31.0,50.52,41.90,68.08,60.92,48.13,42.36,65.62,NaN,NaN,NaN
2500,51.911,46.02,63.85,31.0,50.49,41.97,68.05,61.25,48.33,42.23,65.92,NaN,NaN,NaN
3000,51.997,47.46,63.85,31.4,50.16,41.94,68.03,61.05,48.17,42.15,65.76,NaN,NaN,NaN
3500,51.925,46.68,63.81,31.6,50.23,41.85,68.06,61.12,48.10,42.21,65.59,NaN,NaN,NaN
4000,51.962,46.24,63.67,31.6,50.62,42.18,68.08,61.02,48.09,42.19,65.93,NaN,NaN,NaN
4500,52.082,47.46,63.65,31.3,50.74,41.84,68.11,61.06,48.21,42.61,65.84,NaN,NaN,NaN
5000,51.988,47.01,63.69,31.3,50.55,42.01,68.10,61.23,48.00,42.49,65.50,NaN,NaN,NaN


In [3]:
read_eval_results(
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_mt8400_R2_bz48",
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTH_336px_R2_bz48",
    base_eval_output_dir="evals_master",
    read_format="report_mmai",
    start_step=13000
)

,Average,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
13000,62.894,61.89,69.38,39.8,75.65,51.76,72.73,71.22,61.95,50.82,73.74,NaN,NaN,NaN
13200,62.466,58.67,69.78,38.4,75.84,51.19,72.77,71.44,62.16,50.19,74.22,NaN,NaN,NaN
13400,62.427,59.44,69.04,38.9,75.58,51.18,72.67,71.31,62.23,50.37,73.55,NaN,NaN,NaN
13600,62.653,60.44,70.00,38.3,75.84,52.15,72.81,72.11,61.56,50.23,73.09,NaN,NaN,NaN
13800,62.721,60.67,69.61,39.9,75.52,51.18,72.98,71.60,61.69,50.05,74.01,NaN,NaN,NaN
14000,62.657,60.00,69.73,39.3,75.91,51.41,73.03,71.43,61.83,50.29,73.64,NaN,NaN,NaN
14200,62.539,58.56,70.02,40.3,75.55,51.08,72.93,71.54,62.10,50.31,73.00,NaN,NaN,NaN
14400,62.557,58.22,70.05,40.1,75.65,51.57,72.98,71.77,61.55,50.34,73.34,NaN,NaN,NaN
14600,62.401,58.78,69.84,39.7,76.10,49.98,73.07,71.54,61.89,49.86,73.25,NaN,NaN,NaN
14800,62.461,58.89,69.63,39.0,75.81,51.26,72.98,71.57,61.93,50.06,73.48,NaN,NaN,NaN


In [15]:
read_eval_results(
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_ViTG_392px_mt8400_layer1_R2_bz48",
    # watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_ViTG_392px_mt8400_R2_bz48",
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_mt8400_R2_bz48_resume",
    base_eval_output_dir="evals_master",
    read_format="report_mmai",
    steps=[19000]
)

,Average,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
19000,65.645,61.22,62.47,43.9,82.45,55.94,75.72,75.56,65.34,57.55,76.3,NaN,NaN,NaN


In [36]:
df = read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/MH24_70B_ViTG_392px_mt8400_R2_bz48",
    base_eval_output_dir="evals_master",
    read_format="report_mmai"
)
df.insert(0, 'Average', df.mean(axis=1).round(2))
df

ValueError: cannot insert Average, already exists

In [27]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_ViTG_392px_mt8400_R2_bz48",
    base_eval_output_dir="evals_master",
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
100,48.00,14.81,28.8,61.76,34.10,43.46,12.20,60.73,37.69,19.04,NaN,NaN,NaN
200,48.89,16.49,30.4,63.08,28.68,55.33,14.53,62.00,37.54,51.75,NaN,NaN,NaN
400,52.33,24.86,33.5,65.28,40.23,67.16,55.69,61.20,41.52,65.65,NaN,NaN,NaN
600,54.67,44.58,32.0,66.00,43.53,69.32,61.19,60.28,44.58,67.42,NaN,NaN,NaN
800,52.33,47.15,30.9,68.98,45.00,69.28,64.23,61.37,45.61,65.79,NaN,NaN,NaN
1000,49.78,46.96,32.1,67.52,45.47,69.74,63.97,60.70,45.59,63.62,NaN,NaN,NaN
1200,50.00,49.30,33.1,67.45,46.37,70.39,65.27,60.64,44.87,66.88,NaN,NaN,NaN
1400,49.78,52.98,32.0,68.23,47.14,69.99,64.75,60.31,43.83,65.30,NaN,NaN,NaN
1600,49.44,52.69,32.7,68.17,48.20,69.92,66.32,60.04,46.95,66.44,NaN,NaN,NaN
1800,50.89,52.85,33.4,70.76,48.68,70.38,65.04,60.99,48.53,66.00,NaN,NaN,NaN


In [25]:
read_eval_results(
    watch_path="/fsx_0/checkpoints/mm10.2/MM10.2_Stage1_70B/LLama3.1_70B_BigG_336_R2_bz48",
    base_eval_output_dir="evals_master",
    read_format="report_mmai"
)

,mmmu,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench,POPE_adversarial,POPE_popular,POPE_random
100,49.22,14.92,29.5,61.76,33.77,43.86,12.48,60.80,37.55,21.33,NaN,NaN,NaN
200,47.78,16.15,29.3,62.53,29.51,56.30,15.14,61.62,37.48,48.95,NaN,NaN,NaN
400,52.56,31.34,30.2,64.60,38.76,67.01,54.43,60.96,41.20,61.15,NaN,NaN,NaN
600,53.11,38.53,32.5,66.09,45.55,69.27,62.37,60.07,43.36,64.85,NaN,NaN,NaN
800,51.67,39.28,33.4,69.20,46.36,69.75,62.30,60.41,45.57,66.33,NaN,NaN,NaN
1000,50.33,50.88,33.7,69.11,45.62,70.35,64.97,60.67,43.92,65.81,NaN,NaN,NaN
1200,52.89,52.33,34.6,69.79,47.31,70.13,65.53,60.03,46.65,67.45,NaN,NaN,NaN
1400,50.67,55.33,33.7,69.30,47.71,70.21,65.78,61.14,46.39,67.21,NaN,NaN,NaN
1600,49.56,50.57,32.8,71.44,47.16,70.27,66.53,59.72,46.24,66.00,NaN,NaN,NaN
1800,NaN,NaN,31.5,NaN,49.08,NaN,NaN,NaN,NaN,66.90,NaN,NaN,NaN
